### ST445 Managing and Visualizing Data
#  Using data from the Internet
### Week 4 Lecture, MT 2017 - Kenneth Benoit, Akitaka Matsuo

## Plan for today

- 
- Web-scraping

## Web scraping 

#### What is it?
"Web scraping (web harvesting or web data extraction) is data scraping used for extracting data from websites" [Wikipedia: Web Scraping](https://en.wikipedia.org/wiki/Web_scraping)


![Web Scraping](https://upload.wikimedia.org/wikipedia/commons/d/da/Scrapp.jpg)






## Web-scraping steps
1. Get contents from the web
2. Extract information
3. Reshape and save the information as data

## Get contents from the web

- First of all you need to know where is the information 
- Examples:
    - Government's administrative data
    - Newspaper websites
- The data format
    - web-pages (in html)
    - data files in various format (csv, spss, stata)
    - document files (MS-Word, pdf)
    - API (e.g. JSON)
    - pictures

     


## Get to know the target website

1. Open the website, learn how it's structured
2. "View page source" and "Inspect"


     


## Parse HTML

- 
     


## XPath

- 
     


## R web scraping toolbox

-  get contents
    - `RCurl`
    - `httr`
-  extract information 
    - parsing and analyzing markup language:
        - `XML`
        - `XML2`
    - content extraction with matching 
        - (base R)
        - `stringr`
        - `sgringi`


## Python web scraping toolbox

-  get contents
    - `urllib`
    - `httplib`
    - `requests`
-  extract information 
    - parsing and analyzing markup language:
        - `bs4` (`BeautifulSoup`)
    - content extraction with matching 
        - `re`


## Selenium

- Standard tools for web scraping (e.g. `httr` in R or `urllib`  may not work
- Reasons:
    - "Some websites don’t like to be webscraped. In these cases you may need to disguise your webscraping bot as a human being. Selenium is just the tool for that." [webscraping with Selenium](http://thiagomarzagao.com/2013/11/12/webscraping-with-selenium-part-1/)
    - The information is in non-static contents 
        - [Example 1](http://www.r-datacollection.com/materials/ch-6-ajax/fortunes/fortunes1.html)
        - [Example 2](http://www.r-datacollection.com/materials/ch-6-ajax/fortunes/fortunes2.html)
        - [Example 3](http://www.r-datacollection.com/materials/ch-6-ajax/fortunes/fortunes3.html)
- Solution:
    - Use `selenium` = an automated testing suite for web applications 
    - Manipulate actual web-browser (e.g. Chrome, Firefox) using selenium drivers
    
WIth selenium, you should be able to get whatever you can get with your browser (theoretically speaking...)


## Caveats

#### Web-scraping is not always (or never) welcomed by site-owners

#### Why?
- excessive traffic 
- influence on their revenues

You can be warned, blocked, and even sued. 

#### So, what to do?
1. Read TOC carefully 
2. Check `robot.txt` (c.f. http://www.robotstxt.org/)
2. Get permission if possible 
3. Be nice 
    - place short breaks between fetching
    - scrape during off-peak hours
    - avoid scraping exessive materials



## Further reading

#### Python
*  [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)
*  [Web Scraping with Python](http://shop.oreilly.com/product/0636920034391.do)

#### R
*  [Automated Data Collection with R](http://www.r-datacollection.com/)


## Coming soon

* **Lab**: Simple web-scraping exercise
* **Next week**: API